In [ ]:
!ffmpeg -hide_banner -version

In [ ]:
# Create JSON preview video (draw boxes from sidecar on top of the video)
import json, subprocess
import cv2
from pathlib import Path

# Paths
video_path = Path("~/videos/gun_car.mp4").expanduser()
json_path  = Path("sidecar_tracks.json")
tmp_path   = Path("tmp_preview.mp4")     # intermediate
out_path   = Path("json_preview.mp4")    # final (H.264/yuv420p)

# Load JSON
J = json.loads(json_path.read_text())
frame_map = {int(fr["frame"]): fr for fr in J["frames"]}

# Video setup
cap = cv2.VideoCapture(str(video_path))
assert cap.isOpened(), f"Cannot open {video_path}"
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1e-3:
    fps = float(J.get("fps", 30.0))  # fallback to sidecar or 30

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(tmp_path), fourcc, fps, (w, h))
assert writer.isOpened(), "Failed to open VideoWriter"

# Draw loop
fidx = 0
while True:
    ok, frame = cap.read()
    if not ok:
        break

    fr = frame_map.get(fidx, {"objects": []})
    for o in fr.get("objects", []):
        x, y, bw, bh = o["bbox_xywh"]

        # xywh(center) -> xyxy and clip
        x1 = max(0, int(round(x - bw / 2)))
        y1 = max(0, int(round(y - bh / 2)))
        x2 = min(w - 1, int(round(x + bw / 2)))
        y2 = min(h - 1, int(round(y + bh / 2)))

        # draw
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cls_name = o.get("cls_name")
        cls = o.get("cls")
        tid = o.get("id", "?")
        label = f'{cls_name if cls_name is not None else cls} id={tid}'
        cv2.putText(frame, label, (x1, max(0, y1 - 6)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),
                    1, cv2.LINE_AA)

    writer.write(frame)
    fidx += 1

cap.release()
writer.release()

# Re-encode with ffmpeg to H.264/yuv420p for broad compatibility
subprocess.run([
    "ffmpeg", "-y", "-i", str(tmp_path),
    "-pix_fmt", "yuv420p", "-c:v", "libx264", "-preset", "veryfast", "-crf", "18",
    "-vsync", "cfr", "-r", f"{fps:.3f}", "-movflags", "+faststart",
    str(out_path)
], check=True)

print("Preview saved to", out_path)


In [ ]:
# Create mased out car plate and gun video
import cv2, json, math
from pathlib import Path

# ---- config ----
# video_path = str(Path("~/videos/anpr-demo-video.mp4").expanduser())
json_path  = "sidecar_tracks.json"                          # from Day 1
out_masked = "masked.mp4"
labels_to_blur = None  # e.g., ["face","plate"] if you added cls_name; None = blur all detections
pad_ratio = 0.10       # 10% padding around each box (helps with small drifts)
blur_kernel = 51       # must be odd; larger = stronger blur
sigma = 30

# ---- load json ----
with open(json_path) as f:
    J = json.load(f)

# map frame index -> objects
frame_map = {fr["frame"]: fr["objects"] for fr in J["frames"]}

# ---- open video ----
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Cannot open {video_path}"
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or J.get("fps", 30.0)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(out_masked, fourcc, fps, (W, H))

frame_idx = -1
while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame_idx += 1

    objs = frame_map.get(frame_idx, [])
    for o in objs:
        # optional filter by label name if present
        if labels_to_blur is not None:
            name = o.get("cls_name")
            if name not in labels_to_blur:
                continue

        x, y, w, h = o["bbox_xywh"]  # center-based
        # convert xywh(center) -> xyxy
        x1 = x - w/2; y1 = y - h/2; x2 = x + w/2; y2 = y + h/2

        # padding
        px = w * pad_ratio; py = h * pad_ratio
        x1 -= px; y1 -= py; x2 += px; y2 += py

        # clamp to image bounds
        x1 = int(max(0, math.floor(x1))); y1 = int(max(0, math.floor(y1)))
        x2 = int(min(W-1, math.ceil(x2)));  y2 = int(min(H-1, math.ceil(y2)))
        if x2 <= x1 or y2 <= y1:  # skip degenerate
            continue

        roi = frame[y1:y2, x1:x2]
        # ensure kernel is odd
        k = blur_kernel + (1 - blur_kernel % 2)
        frame[y1:y2, x1:x2] = cv2.GaussianBlur(roi, (k, k), sigma)

    writer.write(frame)

writer.release()
cap.release()
print("Masked video:", out_masked)